In [1]:
import os
import glob

# --- CONFIGURATION ---
INPUT_ROOT = '/kaggle/input/solar-panel-detection-and-identification/PV03'
OUTPUT_DIR = '/kaggle/working/moco_data'

# --- EXECUTION ---
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print("Flattening dataset (Skipping '_label' files)...")

image_count = 0
skipped_count = 0

for root, dirs, files in os.walk(INPUT_ROOT):
    for filename in files:
        # Check if it's an image AND make sure it is NOT a label mask
        if filename.lower().endswith(('.bmp', '.jpg', '.png', '.jpeg', '.tif')):
            
            if '_label' in filename:
                skipped_count += 1
                continue  # SKIP this file

            # --- If we are here, it is a clean image ---
            
            # 1. Full path to the original
            original_path = os.path.join(root, filename)
            
            # 2. Unique name (ParentFolder_ImageName.bmp)
            parent_folder = os.path.basename(root)
            unique_name = f"{parent_folder}_{filename}"
            
            # 3. Create Symlink
            link_path = os.path.join(OUTPUT_DIR, unique_name)
            if not os.path.lexists(link_path):
                os.symlink(original_path, link_path)
                image_count += 1

print(f"Done! Created {image_count} symlinks.")
print(f"Skipped {skipped_count} mask/label files.")

Flattening dataset (Skipping '_label' files)...
Done! Created 2308 symlinks.
Skipped 2308 mask/label files.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageFilter
import random
import os
import glob
import pandas as pd
from tqdm import tqdm
import math

# ==========================================
# 1. CONFIGURATION (The "Fix" Config)
# ==========================================
config = {
    "Dataset Path": "/kaggle/working/moco_data",
    
    # MoCo Hyperparameters
    "Queue Size": 2048,           # 2048 is safe for your dataset size
    "Temperature": 0.07,          # FIXED: Low temp for hard negatives
    "Momentum": 0.999,                # Standard
    
    # Training Setup
    "Training Epochs": 200,           
    "Warmup Epochs": 10,              
    "Batch Size": 64,                 # FIXED: P100 can handle 64 if we strip overhead
    "Learning Rate": 0.03,            # FIXED: Standard LR (no scaling needed for Batch 64)
    "Weight Decay": 1e-4,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Running Fixed MoCo v2 on {device}")

# ==========================================
# 2. AUGMENTATION (Satellite Specific)
# ==========================================
class GaussianBlur(object):
    def __init__(self, sigma=[.1, 2.]):
        self.sigma = sigma
    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = x.filter(ImageFilter.GaussianBlur(radius=sigma))
        return x

moco_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.2, 1.0)),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), # Satellite specific!
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ==========================================
# 3. DATASET
# ==========================================
class SolarPanelDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(root_dir, "*"))
        # Filter labels if any leaked
        self.image_paths = [p for p in self.image_paths if "_label" not in p]
        print(f"✅ Loaded {len(self.image_paths)} clean images.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                q = self.transform(image)
                k = self.transform(image)
            return q, k
        except:
            # Fallback for corrupt images
            return self.__getitem__(random.randint(0, len(self.image_paths)-1))

# ==========================================
# 4. MODEL (The "SimCLR-ified" MoCo)
# ==========================================
class MoCo(nn.Module):
    def __init__(self, base_encoder, dim=128, K=2048, m=0.999, T=0.07):
        super(MoCo, self).__init__()
        self.K = K
        self.m = m
        self.T = T

        # Encoders
        self.encoder_q = base_encoder(num_classes=dim)
        self.encoder_k = base_encoder(num_classes=dim)

        # FIXED: MLP Head with BatchNorm (The Claude Fix)
        # Prevents collapse on small batches
        dim_mlp = self.encoder_q.fc.weight.shape[1]
        self.encoder_q.fc = nn.Sequential(
            nn.Linear(dim_mlp, 2048),
            nn.BatchNorm1d(2048),        # <--- ADDED BN
            nn.ReLU(),
            nn.Linear(2048, dim)
        )
        self.encoder_k.fc = nn.Sequential(
            nn.Linear(dim_mlp, 2048),
            nn.BatchNorm1d(2048),        # <--- ADDED BN
            nn.ReLU(),
            nn.Linear(2048, dim)
        )

        # Initialize Key Encoder
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)
            param_k.requires_grad = False 

        # Queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = nn.functional.normalize(self.queue, dim=0)
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr)
        
        # Replace the keys at ptr
        if ptr + batch_size <= self.K:
            self.queue[:, ptr:ptr + batch_size] = keys.T
        else:
            # Handle wrap around
            rem = self.K - ptr
            self.queue[:, ptr:self.K] = keys[:rem].T
            self.queue[:, :batch_size-rem] = keys[rem:].T
            
        ptr = (ptr + batch_size) % self.K 
        self.queue_ptr[0] = ptr

    def forward(self, im_q, im_k):
        # Query
        q = self.encoder_q(im_q)
        q = nn.functional.normalize(q, dim=1)

        # Key
        with torch.no_grad():
            self._momentum_update_key_encoder()
            k = self.encoder_k(im_k)
            k = nn.functional.normalize(k, dim=1)

        # Logits: Positive (N, 1)
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # Logits: Negative (N, K)
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        # Logits: (N, 1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)
        logits /= self.T

        labels = torch.zeros(logits.shape[0], dtype=torch.long).to(device)
        self._dequeue_and_enqueue(k)
        return logits, labels

# ==========================================
# 5. TRAINING LOOP (No Accumulation)
# ==========================================
dataset = SolarPanelDataset(config['Dataset Path'], transform=moco_transform)

# FIXED: Standard loader, no fancy sampling needed for single GPU
train_loader = DataLoader(
    dataset, 
    batch_size=config['Batch Size'], 
    shuffle=True, 
    drop_last=True, 
    num_workers=4, 
    pin_memory=True
)

model = MoCo(models.resnet50, K=config['Queue Size'], T=config['Temperature']).to(device)
optimizer = optim.SGD(model.parameters(), lr=config['Learning Rate'], momentum=0.9, weight_decay=config['Weight Decay'])
criterion = nn.CrossEntropyLoss()

# Scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['Training Epochs'])

print("🔥 STARTING TRAINING (Fixed Architecture)...")

history = {'loss': [], 'f_std': []}

for epoch in range(config['Training Epochs']):
    model.train()
    running_loss = 0.0
    running_std = 0.0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    
    for images_q, images_k in pbar:
        images_q, images_k = images_q.to(device), images_k.to(device)
        
        optimizer.zero_grad()
        
        logits, labels = model(images_q, images_k)
        loss = criterion(logits, labels)
        
        loss.backward()
        optimizer.step() # Update EVERY step
        
        running_loss += loss.item()
        
        # Monitor Collapse
        with torch.no_grad():
            q_std = nn.functional.normalize(model.encoder_q(images_q), dim=1).std(dim=0).mean().item()
            running_std += q_std
            
        pbar.set_postfix({'loss': loss.item(), 'std': q_std})

    # Epoch Stats
    epoch_loss = running_loss / len(train_loader)
    epoch_std = running_std / len(train_loader)
    scheduler.step()
    
    history['loss'].append(epoch_loss)
    history['f_std'].append(epoch_std)
    
    print(f"📊 Epoch {epoch+1} | Loss: {epoch_loss:.4f} | Feature Std: {epoch_std:.4f}")
    
    # Save checkpoints
    if (epoch+1) % 20 == 0:
        torch.save(model.encoder_q.state_dict(), f"moco_v2_fixed_epoch_{epoch+1}.pth")

# Save final
torch.save(model.encoder_q.state_dict(), "moco_v2_final_fixed.pth")
pd.DataFrame(history).to_csv("training_log_fixed.csv", index=False)
print("🏆 DONE.")

🚀 Running Fixed MoCo v2 on cuda
✅ Loaded 2308 clean images.
🔥 STARTING TRAINING (Fixed Architecture)...


Epoch 1: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, loss=6.61, std=0.0091] 


📊 Epoch 1 | Loss: 5.9292 | Feature Std: 0.0106


Epoch 2: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, loss=6.89, std=0.0124]


📊 Epoch 2 | Loss: 6.7326 | Feature Std: 0.0104


...

📊 Epoch 198 | Loss: 3.7516 | Feature Std: 0.0846


Epoch 199: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, loss=3.95, std=0.0848]


📊 Epoch 199 | Loss: 3.8520 | Feature Std: 0.0846


Epoch 200: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, loss=4.41, std=0.084] 


📊 Epoch 200 | Loss: 3.8457 | Feature Std: 0.0846
🏆 DONE.
